In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm

ImportError: cannot import name 'truncatedpoisson' from 'statsmodels.distributions.discrete' (c:\Users\asus\anaconda3\envs\pt-gpu\lib\site-packages\statsmodels\distributions\discrete.py)